# Goals:
*   You have to implement model *checking* functions for **First Order Logic**, that automatically compute, for any closed formula and any model, the truth value of this formula in this model. (Having completed the model checking section of the first part of the assignment, about Propositional Logic, helps a lot with this part of the assignment.)

# This is part of an assignment
*   This Colab notebook contains the second (and last) part of the assignment.
*   The assignment should be completed in group of two or three students.
*   Students repeating the year ("redoublant·e·s") do not have to complete this assignment (but another one later in the semester).
*   Your grade will depend in part on your code and in part on a short oral examination (a single examination for the whole assignment). To schedule an appointment, follow this link: https://appt.link/timothee-bernard/computational-semantics.
*   Send me an email by Sunday **12th 23:59** to inform me of the composition of your group. By this time, your group should also have scheduled an appointment for the oral examination. Malus: -2 per day of delay.
*   Send me your work (both parts) completed by Sunday  **19th February 23:59**. Malus: -1 per day of delay.
*   Make sure that your code is clear and well commented. **The quality of your code will be taken into account.**
*   **Read all comments and follow all instructions very carefully.** If you do not understand one of them, ask me. Also, remember that everything (each method, its argument·s, etc.) is here for a reason.
*   Any source of inspiration (e.g. the internet, some other student) should be properly acknowledged.

# How to work efficiently with Colab on this project:
*   Copy this notebook (File>Save a copy in Drive).
*   In class, your group can be more efficient if each member works on their own copy: everyone can try to find a solution in parallel before sharing what works with the others.
*   At some point (when most of the problems have been solved), you might want to use a single Colab notebook for the whole group. You can share your copy with your collaborators using the sharing menu.

# How to send me your work:
*   Use the sharing menu (top-right of the window) to share it with timothee.m.r.bernard@gmail.com.
(I don't check this address very often, so, for questions, please use Moodle or my u-paris.fr address.)
*   You are asked to share me both parts of the assignment. (So, two notebooks in total for your group.)
*   Review your code before sharing it with me, in order to check that is is clear, concise and well commented.

# Remark:
*  The code you will find here uses tabulations for indentation. Please be aware of the fact that Python might not behave correctly if you use a mix of tabulations and spaces for indentation. There is a way to set Colab's settings so that the type of characters used for indentation is shown.
*  Make sure that what you print is self-explanatory (one should not have to look at the code to understand what is printed). This advice is relevant to all assignments, for other courses as well as this one.

In [ ]:
# Checks that `o` is an instance of `t` (ex: integer, list).
# Produces a clear error message otherwise.
# This function is not essential but can help a lot for debugging.
def check_type(o, t, name=None):
	if(name is None): name = "[no name]"
	assert isinstance(o, t), (f"Type problem: variable {name} (type: {type(o)}; value: {o}) is not an instance of {t}")

In [ ]:
# Example 1:
check_type([1,2,3], list) # Works fine because [1,2,3] is indeed a list.

In [ ]:
# Example 2:
check_type(1, list) # An AssertionError exception is raised because 1 is not a list.

AssertionError: ignored

*  Although it would be possible to represent constants, variables and predicates with a string only (ex: "Jeanne", "x", "eat"), we here use three different classes for these three kinds of objects so that they can be more easily distinguished from each other.
*  The `isinstance` function can be used to determine whether a given object instantiate a given class.
*  The properties and methods of a class that have a name starting with an underscore ("_"; ex: `self._true_ps`) are not meant to be accessed directly outside of this class, but only within the class itself (in other words, they are *private*). While Python will not say anything special if you do not respect this convention, you definitely should.

In [ ]:
# Constant
class C:
	# name: string
	def __init__(self, name):
		check_type(name, str, "name")
		
		self._name = name
	
	# Defines the behaviour of "==".
	# In this case: two C·s are considered equal if they have the same `_name`.
	def __eq__(self, other):
		return isinstance(other, C) and self._name == other._name

	# Required to be able to use the class in sets or dictionaries.
	def __hash__(self):
		return hash(self._name)
	
	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return self._name
	
	# Returns a string representation of the object. Also used to print the object in a readable way.
	def __repr__(self):
		return str(self)

In [ ]:
# Variable
class V:
	# name: string
	def __init__(self, name):
		check_type(name, str, "name")
		
		self._name = name
	
	# Defines the behaviour of "==".
	# In this case: two V·s are considered equal if they have the same `_name`.
	def __eq__(self, other):
		return isinstance(other, V) and self._name == other._name
	
	# Required to be able to use the class in sets or dictionaries.
	def __hash__(self):
		return hash(self._name)
	
	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return self._name
	
	# Returns a string representation of the object. Also used to print the object in a readable way in more complex case.
	def __repr__(self):
		return str(self)

In [ ]:
# Predicate
class P:
	# name: string
	def __init__(self, name, arity):
		check_type(name, str, "name")
		check_type(arity, int, "arity")
		
		self._name = name
		self.arity = arity
	
	# Defines the behaviour of "==".
	# In this case: two P·s are considered equal if they have the same `_name` and the same `arity`.
	def __eq__(self, other):
		return isinstance(other, P) and (self._name == other._name) and (self.arity == other.arity)
	
	# Required to be able to use the class in sets or dictionaries.
	def __hash__(self):
		return hash((self._name, self.arity))
	
	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return self._name
	
	# Returns a string representation of the object. Also used to print the object in a readable way.
	def __repr__(self):
		return str(self)

# Instructions
*  Instantiate and then print a constant "Sabine", a variable "x" and a binary predicate "eat".
*  Use `isinstance` to check that these objects instantiate the class you think they instantiate.

In [ ]:
sabine = C("Sabine")
x = V("x")
eat = P("eat", 2) # eat(x,y) : x eats y


print(isinstance(sabine, C)) # True
print(isinstance(x, V)) # True
print(isinstance(eat, P)) # True

True
True
True


A model of First Order Logic consists of a domain and an interpretation function.
*  The domain is simply a set the element of which are called "individuals". Here, individuals will be integers.
*  The interpretation function sends any constant to an individual and any predicate to a tuple of individuals (see slides).

In [ ]:
class InterpretationFunc:

	# c_dic: dictionary; keys are C·s, values are integers
	# p_dic: dictionary; keys are P·s, values are sets of tuples of integers
	def __init__(self, c_dic, p_dic):
		self._c_dic = c_dic
		self._p_dic = p_dic
	
	# Remark: __getitem__ can be called using the []-notation: "i[x]" is translated as "i.__getitem__(x)".
	# Returns the interpretation of `x`.
	# x: either a C or a P
	def __getitem__(self, x):
		if(isinstance(x, C)): return self._c_dic[x] # Raises an exception if the constant has no entry in `_c_dic`.
		if(isinstance(x, P)): return self._p_dic.get(x, set()) # Returns an empty set if the predicate has no entry in `_p_dic`.
		raise TypeError
	
	# Returns the list obtained from `l` by replacing all constants by their interpretation (other elements should appear unaffected).
	# (Be aware that this function returns a list and not a tuple. If you need a tuple, use the `tuple` function to convert the list into one.)
	# l: list of C·s and V·s
	def map(self, l):

		check_type(l, list, "l")

		inter_list = list()
		for x in l:
			if isinstance(x, C): # checks if element x is a constant
				inter_list.append(self[x]) # add I(x) to new list

			else:
				inter_list.append(x) # else all other elements as is

		return inter_list


	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		tmp = list(self._c_dic.items())
		tmp.extend(self._p_dic.items())
		s = ', '.join([f"{k}: {v}" for (k, v) in tmp])
		return '{' + s + '}'

# Instructions
*  Complete `InterpretationFunc.map` above.
*  Instantiate an interpretation function `i_func` that interprets the constant "Sabine" as the integer 1, and predicates "tall" "eat" and "like" as non-empty sets. Warning: the one-tuple composed of element `x` is written as `(x,)` in Python (instead of `(x)`, which is just another way to write `x`).
*  Print `i_func`.

In [ ]:
print(type({1}))
d = {}

print(type(d))

<class 'set'>
<class 'dict'>


In [ ]:
y = V("y")
tall = P("tall", 1)
like = P("like", 2)
jamy = C("Jamy")

i_func = InterpretationFunc({sabine: 1, jamy: 2}, {tall: {(1,)}, eat: {(1,3)}, like: {(2,1)}})

print(f"Interpretation function: {i_func}")

Interpretation function: {Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(2, 1)}}


In [ ]:
class Model:
	
	# domain: set of integers
	# i_func: InterpretationFunc
	def __init__(self, domain, i_func):
		check_type(domain, set, "domain")
		check_type(i_func, InterpretationFunc, "i_func")
		
		self.domain = domain
		self.i_func = i_func
	
	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f'{{D={self.domain}; I={self.i_func}}}'

# Instructions
*  Instantiate a model `model` from a finite domain and the `i_func` interpretation function defined previously.
*  Print `model`.

In [ ]:
model = Model({1, 2, 3}, i_func)

print(f"model: {model}")

model: {D={1, 2, 3}; I={Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(2, 1)}}}


* 	We here use partial variable assignment (i.e. that may not assign a value to all variables).
* 	In order to interpret a formula, we will start from an empty variable assignment and then expend/update it when a quantifier is encountered (see the clause for the interpretation of quantifiers in the slides).

# Instructions
*  Complete `VarAssignment.assign`.
*  Complete `VarAssignment.map`.

In [ ]:
# For variable assignments.
class VarAssignment:
	# dic: dictionary; keys are Vs, values are integers
	# If `dic` is not specified, the empty dictionary ({}) is used.
	def __init__(self, dic={}):
		check_type(dic, dict, "dic")
		
		self._dic = dic
	
	# Returns the variable assignment that only differ from the present one (i.e. `self`) with "x := d".
	# The present assignment is not modified and a new assignment is instantiated.
	# x: V
	# d: integer
	def assign(self, x, d):
		check_type(x, V, "x")
		check_type(d, int, "d")
	
		#if x not in self._dic and d not in self._dic.values(): # check if variable x is not already in self._dic
		if d not in self._dic.values():
			new_assign = VarAssignment(self._dic)
			new_assign._dic[x] = d
			return new_assign
	 
		#return self
	
	# Returns the list obtained from `l` by replacing all variables by their assignments (other elements should appear unaffected).
	# (Be aware that this function returns a list and not a tuple. If you need a tuple, use the `tuple` function to convert the list into one.)
	# l: list
	def map(self, l):
		check_type(l, list, "l")
		assign_list = list()

		for x in l:
			# search l for variables and add the assignment of any found to assign_list
			if isinstance(x, V):
				if x in self._dic:
					assign_list.append(self._dic[x])

			# add all other elements unchanged
			else:
				assign_list.append(x)
				
		return assign_list
	
	# Returns a string representation of the object. Used to print the object in a nice way.
	def __str__(self):
		return f'{self._dic}'

*  For this TP, a formula is represented by an instance of the class `Formula` (in fact, of some sub-class of `Formula`).
*  There is one sub-class for each "kind" of formulas, that is to say for each clause in the inductive definition of the language of FOL.

In [ ]:
# The general class for logical formulas.
# This class is sub-classed below.
class Formula:
	# Checks whether the formula is true according to the model `m`.
	# The use of this method requires that the formula be closed.
	# This method does almost nothing by itself. All the work is done by the `check` method defined for each kind of formulas (sub-classes of `Formula`).
	# m: Model
	def check_closed(self, m):
		check_type(m, Model, "m")
		
		f = VarAssignment({}) # Empty partial variable assignment.
		return self.check(m, f)

# Instructions
*  `PredApp` is the sub-class corresponding to formulas composed of a single predicate with all of its arguments (1st clause in the definition of the language of FOL).
*  Complete `PredApp.check`. (The slide about the interpretation of FOL formulas contains all the information you need.)
*  Then, instantiate three formulas (closed or not), using `PredApp` and print  their interpretation according in `model` for some variable assignment (that you have to instantiate).

In [ ]:
# Predicate application
class PredApp(Formula):
	# pred: P
	# args: list of V·s and C·s
	def __init__(self, pred, args):
		check_type(pred, P, "pred")
		assert (pred.arity == len(args)), f"{pred.arity} argument·s expected but {len(args)} given."
		check_type(args, list, "args")
		
		self._pred = pred
		self._args = args
	
	# Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
	# f: VarAssignment
	def check(self, m, f):
		check_type(m, Model, "m")
		check_type(f, VarAssignment, "f")
		#print(m.i_func[self._pred])
		
		# create list that replaces all variables in the args list with an individual of the domain according to its variable assignment
		# i.e. f[x := d](x) = d
		#mappings = tuple(f.map(self._args))
		c_mappings = m.i_func.map(self._args) # replacing constants with I(c) (i.e. integers member of the domain)
		v_mappings = tuple(f.map(c_mappings)) # replacing variables with f[x := d](x)
		
		#print(f"I(P) = {m.i_func[self._pred]} type = {type(m.i_func[self._pred])}")
		#print(f"mappings = {v_mappings}")
		# return True if the tuple (t1, t2...tn) is a member of I(P)
		if v_mappings in m.i_func[self._pred]:
			return True

		return False



	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f"{self._pred}({','.join([str(x) for x in self._args])})"

In [ ]:
i_func = InterpretationFunc({sabine: 1, jamy: 2}, {tall: {(1,)}, eat: {(1,3)}, like: {(1,2)}})
model = Model({1, 2, 3}, i_func)
var_assign = VarAssignment({x: 1, y: 2})

print(f"i_func: {i_func}\n")
print(f"model: {model}\n")
print(f"variable assignment: {var_assign}")
print("-----------------------------------------------------------------------\n")

x_is_tall = PredApp(tall, [x]) # ¬tall(x)
print(f"⟦{x_is_tall}⟧M,f = {x_is_tall.check(model, var_assign)}\n") # True

jamy_is_tall = PredApp(tall, [jamy])
print(f"⟦{jamy_is_tall}⟧M,f = {jamy_is_tall.check(model, var_assign)}\n") # False

x_likes_y = PredApp(like, [x, y])
print(f"⟦{x_likes_y}⟧M,f = {x_likes_y.check(model, var_assign)}\n") # True

sabine_is_tall = PredApp(tall, [sabine])
print(f"⟦{sabine_is_tall}⟧M,f = {sabine_is_tall.check(model, var_assign)}\n") #True

i_func: {Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}

model: {D={1, 2, 3}; I={Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}}

variable assignment: {x: 1, y: 2}
-----------------------------------------------------------------------

⟦tall(x)⟧M,f = True

⟦tall(Jamy)⟧M,f = False

⟦like(x,y)⟧M,f = True

⟦tall(Sabine)⟧M,f = True



# Instructions
*  Write `Neg`, a sub-class of `Formula`, for negation.
*  This class must possess, in addition to a constructor (`__init__`), a `check` method and a `__str__` one (see `PredApp`; you can use the ¬ symbol in `__str__`).
*  (The slide about the syntax of FOL and the one about its semantics contain all the information you need.)
*  Use `check_type` at the beginning of each method in order to check the type of each argument.
*  Instantiate several formulas using `Neg`, display these formulas and there value in some model for some variable assignment.

In [ ]:
# Negation
class Neg(Formula):
  # phi: Formula
	def __init__(self, phi):
		check_type(phi, Formula, "phi")
		self._phi = phi
	
	
	# Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
  # f: VarAssignment
	def check(self, m, f):
		check_type(m, Model, "m")
		check_type(f, VarAssignment, "f")

		
		# return True if the formula is False and False otherwise
		return not self._phi.check(m, f)
	
	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f'(¬{self._phi})'

In [ ]:
i_func = InterpretationFunc({sabine: 1, jamy: 2}, {tall: {(1,)}, eat: {(1,3)}, like: {(1,2)}})
model = Model({1, 2, 3}, i_func)
var_assign = VarAssignment({x: 1, y: 2})

print(f"i_func: {i_func}\n")
print(f"model: {model}\n")
print(f"variable assignment: {var_assign}")
print("-----------------------------------------------------------------------\n")

neg_x_is_tall = Neg(PredApp(tall, [x])) # ¬tall(x)
print(f"⟦{neg_x_is_tall}⟧M,f = {neg_x_is_tall.check(model, var_assign)}\n") # False

neg_jamy_is_tall = Neg(PredApp(tall, [jamy]))
print(f"⟦{neg_jamy_is_tall}⟧M,f = {neg_jamy_is_tall.check(model, var_assign)}\n") # True

neg_x_likes_y = Neg(PredApp(like, [x, y]))
print(f"⟦{neg_x_likes_y}⟧M,f = {neg_x_likes_y.check(model, var_assign)}\n") # False

i_func: {Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}

model: {D={1, 2, 3}; I={Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}}

variable assignment: {x: 1, y: 2}
-----------------------------------------------------------------------

⟦(¬tall(x))⟧M,f = False

⟦(¬tall(Jamy))⟧M,f = True

⟦(¬like(x,y))⟧M,f = False



# Instructions
*  Same instructions for `Ex`, a sub-class for existential quantification.
*  (You can use the ∃ symbol in `__str__`.)

In [ ]:
import random
# existential quantification
class Ex(Formula):
# phi: Formula
	def __init__(self, x, phi):
   
		check_type(x, V, "x")
		check_type(phi, Formula, "phi")
  
		self._x = x
		self._phi = phi 
	# Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
	# f: VarAssignment
	def check(self, m, f):
		check_type(m, Model, "m")
		check_type(f, VarAssignment, "f")

		#print(f"domain = {m.domain}")
		#new_var_assign = VarAssignment({})	

		for d in m.domain:
			#print(f"d = {d}")
			#var_assign_x_to_d = f.assign(self._x, d)
			new_f = f.assign(self._x, d) # update var assignment
      # if there exists some d in the domain of the model such that the interpretation of the formula phi according to the model `m`
      # and variable assignment f[x := d] return True
			print(f"VarAssign: {f}")
			if self._phi.check(m, f):
				return True

    # otherwise return False
		return False

	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f'(∃{self._x}. {self._phi})'

In [ ]:
tup1 = (1,)
tup2 = (1,)

set1 = {(1,)}

set1

var_assign = VarAssignment()
var_assign.assign(x, 1)
print(var_assign)

{x: 1}


In [ ]:
i_func = InterpretationFunc({sabine: 1, jamy: 2}, {tall: {(1,)}, eat: {(1,3)}, like: {(1,2)}})
model = Model({1, 2, 3}, i_func)
empty_var_assign = VarAssignment({})#VarAssignment({x: 1})

print(f"i_func: {i_func}\n")
print(f"model: {model}\n")
print(f"variable assignment: {empty_var_assign}")
print("-----------------------------------------------------------------------\n")

exists_x_x_tall = Ex(x, x_is_tall) # (∃x. tall(x))
print(f"⟦{exists_x_x_tall}⟧M,f = {exists_x_x_tall.check(model, empty_var_assign)}\n")

exists_x_x_likes_y = Ex(x, x_likes_y)
print(f"⟦{exists_x_x_likes_y}⟧M,f = {exists_x_x_likes_y.check(model, empty_var_assign)}\n")

exists_x_x_likes_x = Ex(x, PredApp(like, [x,x]))
print(f"⟦{exists_x_x_likes_x}⟧M,f = {exists_x_x_likes_x.check(model, empty_var_assign)}\n")

exists_x_exists_y_x_likes_y = Ex(y, exists_x_x_likes_y)
print(f"⟦{exists_x_exists_y_x_likes_y}⟧M,f = {exists_x_exists_y_x_likes_y.check(model, empty_var_assign)}")

#print(exists_x_tall.check(model, var_assign_x))
#print(exists_y_tall.check(model, var_assign_y))

i_func: {Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}

model: {D={1, 2, 3}; I={Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}}

variable assignment: {}
-----------------------------------------------------------------------

VarAssign: {x: 1}
⟦(∃x. tall(x))⟧M,f = True

VarAssign: {x: 1}
VarAssign: {x: 2}
VarAssign: {x: 3}
⟦(∃x. like(x,y))⟧M,f = False

VarAssign: {x: 1}
VarAssign: {x: 2}
VarAssign: {x: 3}
⟦(∃x. like(x,x))⟧M,f = False

VarAssign: {x: 3, y: 1}
VarAssign: {x: 3, y: 1}
VarAssign: {x: 2, y: 1}
VarAssign: {x: 3, y: 1}
VarAssign: {x: 3, y: 2}
VarAssign: {x: 1, y: 2}
⟦(∃y. (∃x. like(x,y)))⟧M,f = True


# Instructions
*  Same instructions for `Conj`, a sub-class for conjunction.
*  (You can use the ∧ symbol in `__str__`)

In [ ]:
# conjunction
class Conj(Formula):

  # phi: Formula
  # psi: Formula
	def __init__(self, phi, psi):
		check_type(phi, Formula, "phi")
		check_type(psi, Formula, "psi")
		
		self._phi = phi
		self._psi = psi

  # Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
	# f: VarAssignment

	def check(self, m, f):
		check_type(m, Model, "m")
		check_type(f, VarAssignment, "f")
  

		return self._phi.check(m, f) and self._psi.check(m, f)

  # Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f"({self._phi} ∧ {self._psi})"

In [ ]:
i_func = InterpretationFunc({sabine: 1, jamy: 2}, {tall: {(1,)}, eat: {(1,3)}, like: {(1,2)}})
model = Model({1, 2, 3}, i_func)
var_assign = VarAssignment({})

print(f"i_func: {i_func}\n")
print(f"model: {model}\n")
print(f"variable assignment: {var_assign}")
print("-----------------------------------------------------------------------\n")

y_is_tall = PredApp(tall, [y])

x_is_tall_and_y_is_tall = (Conj(x_is_tall, y_is_tall))
print(f"⟦{x_is_tall_and_y_is_tall}⟧M,f = {x_is_tall_and_y_is_tall.check(model, var_assign)}")

exists_x_is_tall_and_x_likes_y = Ex(x, Ex(y, Conj(x_is_tall, x_likes_y)))
print(f"⟦{exists_x_is_tall_and_x_likes_y}⟧M,f = {exists_x_is_tall_and_x_likes_y.check(model, var_assign)}\n") # True

i_func: {Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}

model: {D={1, 2, 3}; I={Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}}

variable assignment: {}
-----------------------------------------------------------------------

⟦(tall(x) ∧ tall(y))⟧M,f = False
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
⟦(∃x. (∃y. (tall(x) ∧ like(x,y))))⟧M,f = True



# Instructions
*   Test `check_closed` using several complex and diversed closed formulas.

In [ ]:
# defining some closed formulas
print("--Model--")
pasta = C("pasta")
i_func = InterpretationFunc({sabine: 1, jamy: 2}, {tall: {(1,)}, eat: {(1,3)}, like: {(1,2)}})
model = Model({1, 2, 3}, i_func)
print(model)
x_likes_y = PredApp(like, [x, y]) # like(x,y)

exists_y_x_likes_y = Ex(y, x_likes_y) # (∃y. like(x,y))
exists_x_y_x_likes_y = Ex(x, exists_y_x_likes_y) # (∃x. (∃y. like(x,y)))
x_eats_y = PredApp(eat, [x,y]) # eat(x,y)

print("\n--Formulas--")
print(exists_x_y_x_likes_y)

print(exists_x_y_x_likes_y.check_closed(model)) # True
print()
print(Ex(x, Ex(y, Conj(x_is_tall, x_likes_y)))) # (∃x. (∃y. (tall(x) ∧ like(x,y))))
print(Ex(x, Ex(y, Conj(x_is_tall, (x_likes_y)))).check_closed(model)) # true
print()
print(Ex(x, Ex(y, Conj(y_is_tall, x_likes_y)))) # (∃x. (∃y. (tall(y) ∧ like(x,y))))
print(Ex(x, Ex(y, Conj(y_is_tall, x_likes_y))).check_closed(model)) # false
print()
print(Ex(x, Ex(y,x_eats_y))) # (∃x. (∃y. eat(x,y)))
print(Ex(x, Ex(y,x_eats_y)).check_closed(model)) # true

print(Ex(x, Ex(y, Conj(Neg(y_is_tall), x_likes_y)))) # (∃x. (∃y. (¬tall(y) ∧ like(x,y))))
print(Ex(x, Ex(y, Conj(Neg(y_is_tall), x_likes_y))).check_closed(model)) # true

print(Neg(Ex(x, (Ex(y,x_eats_y))))) # (¬(∃x. (∃y. eat(x,y))))
print(Neg(Ex(x, (Ex(y,x_eats_y)))).check_closed(model)) # false
print()
print(Ex(x, Ex(y, Conj(x_is_tall, Neg(x_likes_y))))) # (∃x. (∃y. (tall(x) ∧ ¬like(x,y))))
print(Ex(x, Ex(y, Conj(x_is_tall, Neg(x_likes_y)))).check_closed(model)) # false


--Model--
{D={1, 2, 3}; I={Sabine: 1, Jamy: 2, tall: {(1,)}, eat: {(1, 3)}, like: {(1, 2)}}}

--Formulas--
(∃x. (∃y. like(x,y)))
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
True

(∃x. (∃y. (tall(x) ∧ like(x,y))))
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
True

(∃x. (∃y. (tall(y) ∧ like(x,y))))
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
VarAssign: {x: 1, y: 3}
VarAssign: {x: 2, y: 3}
VarAssign: {x: 2, y: 1}
VarAssign: {x: 2, y: 1}
VarAssign: {x: 2, y: 3}
VarAssign: {x: 2, y: 3}
VarAssign: {x: 2, y: 1}
VarAssign: {x: 2, y: 1}
VarAssign: {x: 2, y: 3}
False

(∃x. (∃y. eat(x,y)))
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
VarAssign: {x: 1, y: 3}
True
(∃x. (∃y. ((¬tall(y)) ∧ like(x,y))))
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
True
(¬(∃x. (∃y. eat(x,y))))
VarAssign: {x: 1}
VarAssign: {x: 1}
VarAssign: {x: 1, y: 2}
VarAssign: {x: 1, y: 3}
False

(∃x. (∃y. (tall(x) ∧ (¬like(x,y)))))
VarAssign: {x: 1}
VarAss